## Gaming with Min-Max Algorithm - Solution template

# Things to follow

1. Use appropriate data structures to represent the graph using python libraries
2. Provide proper documentation
3. Create neat solution without error during game playing

### Coding begins here

### PEAS - Data structures and fringes that define the Agent environment goes here


**Performance Measure:**

The performance measure for the players (agents) in this game is to maximize their final sum of chosen numbers while adhering to the game rules.

**Environment:**

The environment consists of the set of natural numbers, {1, 2, 3, …, n}, where 'n' is agreed upon before the game.

The environment also includes the current state of the game, which is represented by the remaining numbers in the set, the sum chosen by each player, and whose turn it is (P1 or P2).

**Actuators:**

The actuators in this game are the actions taken by each player to select one or more numbers from the remaining set of numbers.

**Sensors**:

The sensors are used to perceive the current state of the game, including the remaining numbers, the current sums chosen by each player, and whose turn it is.

In [ ]:
#Definitions:

def game_over(state):
    # Check if the game is over
    # In Catch-Up, the game is over when there are no remaining numbers to choose
    return len(state['remaining_numbers']) == 0

def possible_actions(state):
    # Generate possible actions for the current state
    # In Catch-Up, the actions are choosing one of the remaining numbers
    return state['remaining_numbers']

def result(state, action):
    # Apply the action to the current state and return the new state
    # In Catch-Up, update the chosen number and switch the current player
    new_state = state.copy()
    new_state['remaining_numbers'].remove(action)
    if state['current_player'] == 'p1':
        new_state['p1_sum'] += action
        new_state['current_player'] = 'p2'
    else:
        new_state['p2_sum'] += action
        new_state['current_player'] = 'p1'
    return new_state

def evaluate_state(state):
    # Evaluate the current state
    # In Catch-Up, calculate the difference between players' sums
    return state['p1_sum'] - state['p2_sum']

In [ ]:
def choose_best_action_alpha_beta(state, depth, alpha, beta):
    best_eval = -float('inf')
    best_action = None
    for action in possible_actions(state):
        eval = alpha_beta(result(state, action), depth - 1, alpha, beta, False)
        if eval > best_eval:
            best_eval = eval
            best_action = action
        alpha = max(alpha, eval)
        if beta <= alpha:
            break
    return best_action

### Implementation of the Min-Max algorithm

In [ ]:
#The Min-Max algorithm can be used to find the optimal strategy for a player in this game. Here's a Python code snippet for implementing the Min-Max algorithm:

def min_max(state, depth, is_maximizer):
    if depth == 0 or game_over(state):
        return evaluate_state(state)

    if is_maximizer:
        max_eval = -float('inf')
        for action in possible_actions(state):
            eval = min_max(result(state, action), depth - 1, False)
            max_eval = max(max_eval, eval)
        return max_eval
    else:
        min_eval = float('inf')
        for action in possible_actions(state):
            eval = min_max(result(state, action), depth - 1, True)
            min_eval = min(min_eval, eval)
        return min_eval

def choose_best_action(state, depth):
    best_eval = -float('inf')
    best_action = None
    for action in possible_actions(state):
        eval = min_max(result(state, action), depth - 1, False)
        if eval > best_eval:
            best_eval = eval
            best_action = action
    return best_action

# Example usage:
initial_state = {
    'remaining_numbers': [1, 2, 3, 4, 5],
    'p1_sum': 0,
    'p2_sum': 0,
    'current_player': 'p1'
}
best_action = choose_best_action(initial_state, depth=3)
print("Best action:", best_action)

Best action: 1


### Implementation of the alpha-beta pruning  

In [ ]:

#Alpha-Beta pruning can be applied to optimize the Min-Max algorithm. Here's a Python code snippet for implementing Alpha-Beta pruning:

def alpha_beta(state, depth, alpha, beta, is_maximizer):
    if depth == 0 or game_over(state):
        return evaluate_state(state)

    if is_maximizer:
        max_eval = -float('inf')
        for action in possible_actions(state):
            eval = alpha_beta(result(state, action), depth - 1, alpha, beta, False)
            max_eval = max(max_eval, eval)
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return max_eval
    else:
        min_eval = float('inf')
        for action in possible_actions(state):
            eval = alpha_beta(result(state, action), depth - 1, alpha, beta, True)
            min_eval = min(min_eval, eval)
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return min_eval

# Example usage:
initial_state = {
    'remaining_numbers': [1, 2, 3, 4, 5],
    'p1_sum': 0,
    'p2_sum': 0,
    'current_player': 'p1'
}
best_action = choose_best_action_alpha_beta(initial_state, depth=3, alpha=-float('inf'), beta=float('inf'))
print("Best action:", best_action)


Best action: 1


### Choice and implementation of the Static Evaluation Function.

In [ ]:
#Code block - Start the game

import random

def choose_subset(player_sum, opponent_sum, available_numbers):
    # Function to choose a subset of numbers that brings player_sum at least up to opponent_sum
    possible_choices = []
    for num in available_numbers:
        temp_sum = player_sum
        temp_choices = []
        for n in sorted(available_numbers, reverse=True):
            if temp_sum + n >= opponent_sum:
                temp_choices.append(n)
                temp_sum += n
                possible_choices.append(temp_choices)
                break
            elif temp_sum + n < opponent_sum:
                temp_choices.append(n)
                temp_sum += n
    return random.choice(possible_choices)

def play_game(n):
    available_numbers = list(range(1, n+1))
    p1_sum, p2_sum = 0, 0

    # P1's first move
    p1_choice = random.choice(available_numbers)
    p1_sum += p1_choice
    available_numbers.remove(p1_choice)
    print(f"P1 chooses {p1_choice}. Sum: {p1_sum}")

    while available_numbers:
        # P2's turn
        p2_choices = choose_subset(p2_sum, p1_sum, available_numbers)
        p2_sum += sum(p2_choices)
        print(f"P2 chooses {p2_choices}. Sum: {p2_sum}")
        for choice in p2_choices:
            available_numbers.remove(choice)

        if not available_numbers:
            break

        # P1's turn
        p1_choices = choose_subset(p1_sum, p2_sum, available_numbers)
        p1_sum += sum(p1_choices)
        print(f"P1 chooses {p1_choices}. Sum: {p1_sum}")
        for choice in p1_choices:
            available_numbers.remove(choice)

    # Determine winner
    if p1_sum > p2_sum:
        print(f"P1 wins with a sum of {p1_sum}!")
    elif p2_sum > p1_sum:
        print(f"P2 wins with a sum of {p2_sum}!")
    else:
        print("It's a tie!")

# The game starts with numbers from 1 to 5
play_game(4)

P1 chooses 1. Sum: 1
P2 chooses [4]. Sum: 4
P1 chooses [3]. Sum: 4
P2 chooses [2]. Sum: 6
P2 wins with a sum of 6!
